# pandas的拼接操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

## 0. 回顾numpy的级联

In [1]:
import numpy as np

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

In [2]:
nd1 = np.random.randint(0,10,size=(3,3))
nd2 = np.random.randint(0,10,size=(3,3))
print(nd1)
print(nd2)

[[8 2 4]
 [3 6 9]
 [3 4 1]]
[[7 4 6]
 [4 4 5]
 [9 9 4]]


In [5]:
np.concatenate([nd1,nd2],axis=1)

array([[8, 2, 4, 7, 4, 6],
       [3, 6, 9, 4, 4, 5],
       [3, 4, 1, 9, 9, 4]])

In [6]:
nd3 = np.random.randint(0,10,size=3)
nd3

array([3, 5, 4])

In [7]:
np.concatenate([nd1,nd3]) # 维度不一致，无法级联

ValueError: all the input arrays must have same number of dimensions

In [8]:
nd4 = np.random.randint(0,10,size=(3,2))
nd4

array([[4, 0],
       [8, 0],
       [5, 8]])

In [10]:
np.concatenate([nd1,nd4],axis=1)

array([[8, 2, 4, 4, 0],
       [3, 6, 9, 8, 0],
       [3, 4, 1, 5, 8]])

In [11]:
import pandas as pd
from pandas import DataFrame,Series

In [23]:
# 为了后面创建数据表方便，定义一个函数，用于创建DataFrame
def make_df(rows,cols):
    
    data = {c:[c+str(i) for i in rows] for c in cols}
    
    return DataFrame(data,index=rows,columns=cols)

In [24]:
make_df([1,2,3],["a","b"])

,a,b
1,a1,b1
2,a2,b2
3,a3,b3


In [25]:
df1 = make_df([1,2,3],list("abc"))
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [26]:
df2 = make_df([4,5,6],list("abc"))
df2

,a,b,c
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


## 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

### 1)  简单级联

In [27]:
pd.concat([df1,df2],axis=0)

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


In [28]:
# 列级联
pd.concat([df1,df2],axis=1) # df的级联遵从自动补全机制

,a,b,c,a,b,c
1,a1,b1,c1,NaN,NaN,NaN
2,a2,b2,c2,NaN,NaN,NaN
3,a3,b3,c3,NaN,NaN,NaN
4,NaN,NaN,NaN,a4,b4,c4
5,NaN,NaN,NaN,a5,b5,c5
6,NaN,NaN,NaN,a6,b6,c6


In [30]:
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [29]:
df3 = make_df([1,2,4],list("bcd"))
df3

,b,c,d
1,b1,c1,d1
2,b2,c2,d2
4,b4,c4,d4


In [31]:
pd.concat([df1,df3],axis=0) # 行级联补列

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
1,NaN,b1,c1,d1
2,NaN,b2,c2,d2
4,NaN,b4,c4,d4


In [32]:
pd.concat([df1,df3],axis=1)

,a,b,c,b,c,d
1,a1,b1,c1,b1,c1,d1
2,a2,b2,c2,b2,c2,d2
3,a3,b3,c3,NaN,NaN,NaN
4,NaN,NaN,NaN,b4,c4,d4


解决行标（或者列标）冲突的问题

In [33]:
#1) 选择ignore_index为忽略索引，索引将从0开始重新排列
pd.concat([df1,df3],axis=0,ignore_index=True)

,a,b,c,d
0,a1,b1,c1,NaN
1,a2,b2,c2,NaN
2,a3,b3,c3,NaN
3,NaN,b1,c1,d1
4,NaN,b2,c2,d2
5,NaN,b4,c4,d4


In [34]:
#2) 选择使用多层索引
pd.concat([df1,df3],axis=0,keys=["小名","小黄"])


a   b   c    d
小名 1   a1  b1  c1  NaN
   2   a2  b2  c2  NaN
   3   a3  b3  c3  NaN
小黄 1  NaN  b1  c1   d1
   2  NaN  b2  c2   d2
   4  NaN  b4  c4   d4

============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

============================================

### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

In [37]:
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [38]:
df4 = make_df([3,4,5,6],list("abcd"))
df4

,a,b,c,d
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


1） 外连接

In [40]:
pd.concat([df1,df4],axis=0,join="outer")

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


In [41]:
pd.concat([df1,df2],axis=1,join="outer")
# join代表连接方式，默认是outer，即外连接
# 对不一致的地方进行补NaN

,a,b,c,a,b,c
1,a1,b1,c1,NaN,NaN,NaN
2,a2,b2,c2,NaN,NaN,NaN
3,a3,b3,c3,NaN,NaN,NaN
4,NaN,NaN,NaN,a4,b4,c4
5,NaN,NaN,NaN,a5,b5,c5
6,NaN,NaN,NaN,a6,b6,c6


2）内连接

In [44]:
pd.concat([df1,df4],axis=0,join="inner")

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3
3,a3,b3,c3
4,a4,b4,c4
5,a5,b5,c5
6,a6,b6,c6


In [46]:
pd.concat([df1,df4],axis=1,join="inner")
# join为inner的时候是内连接，把索引不一致的地方剔除掉

,a,b,c,a,b,c,d
3,a3,b3,c3,a3,b3,c3,d3


- 连接指定轴 join_axes

In [47]:
df5 = make_df([5,6,7,8],list("abde"))
df5

,a,b,d,e
5,a5,b5,d5,e5
6,a6,b6,d6,e6
7,a7,b7,d7,e7
8,a8,b8,d8,e8


In [54]:
pd.concat([df1,df4,df5],join_axes=[pd.Index(["a","e"])])

,a,e
1,a1,NaN
2,a2,NaN
3,a3,NaN
3,a3,NaN
4,a4,NaN
5,a5,NaN
6,a6,NaN
5,a5,e5
6,a6,e6
7,a7,e7


In [60]:
pd.concat([df1,df4,df5],axis=1,join_axes=[df1.index])

,a,b,c,a,b,c,d,a,b,d,e
1,a1,b1,c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a2,b2,c2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a3,b3,c3,a3,b3,c3,d3,NaN,NaN,NaN,NaN


In [67]:
pd.concat([df1,df2,df3,df4],axis=0,join_axes=[pd.Index(["d"])])

,d
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
1,d1
2,d2
4,d4
3,d3


============================================

练习14：

    假设【期末】考试ddd2的成绩没有张三的，只有李四、王老五、赵小六的，使用多种方法级联

============================================

### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

列表的append()

In [68]:
l = [1,2,3,4]
l

[1, 2, 3, 4]

In [70]:
l.append(9)
l

[1, 2, 3, 4, 9, 9]

In [71]:
l.append("123")

In [73]:
l.append(["a","b"])
l

[1, 2, 3, 4, 9, 9, '123', ['a', 'b']]

In [74]:
df1

,a,b,c
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [82]:
df1.append("123") # 常数无法级联，也就无法进行append

TypeError: cannot concatenate a non-NDFrame object

In [76]:
df1.append([12,2,3])
# 如果追加一个列表，会把这个列表直接创建成一个DataFrame，进一步的append

,a,b,c,0
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
0,NaN,NaN,NaN,12.0
1,NaN,NaN,NaN,2.0
2,NaN,NaN,NaN,3.0


In [78]:
df4

,a,b,c,d
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


In [84]:
df1.append(df4)
# DataFrame的append相当于进行行级联

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

###  1) 一对一合并

公共属性中，如果对应的值是一一对应关系，我们称之为一对一合并

In [85]:
df1 = DataFrame({
    "name":["小明","小黄","小绿","小白"],
    "age":[18,28,30,40],
    "job":["student","driver","seller","boss"]
},index=list("abcd"))
df1

,age,job,name
a,18,student,小明
b,28,driver,小黄
c,30,seller,小绿
d,40,boss,小白


In [86]:
df2 = DataFrame({
    "name":["小明","小黄","小绿","小白"],
    "home":["北京","上海","杭州","新加坡"],
    "phone":[110,120,119,999]
},index=list("abcd"))
df2

,home,name,phone
a,北京,小明,110
b,上海,小黄,120
c,杭州,小绿,119
d,新加坡,小白,999


In [88]:
pd.concat([df1,df2],axis=1)

,age,job,name,home,name,phone
a,18,student,小明,北京,小明,110
b,28,driver,小黄,上海,小黄,120
c,30,seller,小绿,杭州,小绿,119
d,40,boss,小白,新加坡,小白,999


In [89]:
pd.merge(df1,df2)

,age,job,name,home,phone
0,18,student,小明,北京,110
1,28,driver,小黄,上海,120
2,30,seller,小绿,杭州,119
3,40,boss,小白,新加坡,999


In [92]:
df3 = DataFrame({
    "姓名":["小明","小黄","小绿","小白"],
    "home":["北京","上海","杭州","新加坡"],
    "phone":[110,120,119,999]
},index=list("abcd"))
df3

,home,phone,姓名
a,北京,110,小明
b,上海,120,小黄
c,杭州,119,小绿
d,新加坡,999,小白


In [93]:
pd.merge(df1,df3)

MergeError: No common columns to perform merge on

In [95]:
df1.merge(df2,axis=0) # merge没有这个参数，合并针对的是列

TypeError: merge() got an unexpected keyword argument 'axis'

【注意】

    1）合并不能操作列，针对于行
    2）合并的条件：要有公共的属性（即列）
    3）合并的时候把公共的属性对应的值并在一起

### 2) 多对一合并

In [96]:
df3 = DataFrame({
    "name":["小明","小黄","小绿","小白","小明","小绿","小明","小黄"],
    "home":["北京","上海","杭州","新加坡","日本","韩国","美国","深圳"],
    "phone":[110,120,119,999,10086,10010,4004000,123456]
},index=list("abcdefgh"))
df3

,home,name,phone
a,北京,小明,110
b,上海,小黄,120
c,杭州,小绿,119
d,新加坡,小白,999
e,日本,小明,10086
f,韩国,小绿,10010
g,美国,小明,4004000
h,深圳,小黄,123456


In [97]:
df1

,age,job,name
a,18,student,小明
b,28,driver,小黄
c,30,seller,小绿
d,40,boss,小白


In [98]:
df1.merge(df3)

,age,job,name,home,phone
0,18,student,小明,北京,110
1,18,student,小明,日本,10086
2,18,student,小明,美国,4004000
3,28,driver,小黄,上海,120
4,28,driver,小黄,深圳,123456
5,30,seller,小绿,杭州,119
6,30,seller,小绿,韩国,10010
7,40,boss,小白,新加坡,999


一对多：在某个公共属性中表1中的某个值，在表2中有多个和他对应

合并规则：把表1中对应的那些行复制到表2的数量，然后对应合并

### 3) 多对多合并

In [102]:
df3

,home,name,phone
a,北京,小明,110
b,上海,小黄,120
c,杭州,小绿,119
d,新加坡,小白,999
e,日本,小明,10086
f,韩国,小绿,10010
g,美国,小明,4004000
h,深圳,小黄,123456


In [101]:
df4 = DataFrame({
    "name":["小明","小黄","小绿","小白","小明","小黄","小白"],
    "age":[18,28,30,40,20,18,25],
    "job":["student","driver","seller","boss","医生","演员","男演员"]
},index=list("abcdefg"))
df4

,age,job,name
a,18,student,小明
b,28,driver,小黄
c,30,seller,小绿
d,40,boss,小白
e,20,医生,小明
f,18,演员,小黄
g,25,男演员,小白


In [103]:
df3.merge(df4)

,home,name,phone,age,job
0,北京,小明,110,18,student
1,北京,小明,110,20,医生
2,日本,小明,10086,18,student
3,日本,小明,10086,20,医生
4,美国,小明,4004000,18,student
5,美国,小明,4004000,20,医生
6,上海,小黄,120,28,driver
7,上海,小黄,120,18,演员
8,深圳,小黄,123456,28,driver
9,深圳,小黄,123456,18,演员


多对多：表1中有多个相同值，表2中和这个值相同的也有多个

合并：分别拿出表1中所有的值，去和表2中对应的值匹配生成一个新的行，最终生成的数据个条数是两者的乘积

### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用

In [106]:
df1

,age,job,name
a,18,student,小明
b,28,driver,小黄
c,30,seller,小绿
d,40,boss,小白


In [105]:
df5 = DataFrame({
    "name":["小明","小黄","小绿","小白","小明","小绿","小明","小黄"],
    "home":["北京","上海","杭州","新加坡","日本","韩国","美国","深圳"],
    "phone":[110,120,119,999,10086,10010,4004000,123456],
    "age":[18,19,20,21,22,23,24,25]
},index=list("abcdefgh"))
df5

,age,home,name,phone
a,18,北京,小明,110
b,19,上海,小黄,120
c,20,杭州,小绿,119
d,21,新加坡,小白,999
e,22,日本,小明,10086
f,23,韩国,小绿,10010
g,24,美国,小明,4004000
h,25,深圳,小黄,123456


In [109]:
df4.merge(df5)
# 现在有两个公共的属性，在合并的时候要参考两个key值

,age,job,name,home,phone
0,18,student,小明,北京,110


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

In [111]:
# 如果有多个相同的属性，我们可以制定以哪几个属性为基准进行合并
df4.merge(df5,on="name",suffixes=["_真实","_虚假"])

,age_真实,job,name,age_虚假,home,phone
0,18,student,小明,18,北京,110
1,18,student,小明,22,日本,10086
2,18,student,小明,24,美国,4004000
3,20,医生,小明,18,北京,110
4,20,医生,小明,22,日本,10086
5,20,医生,小明,24,美国,4004000
6,28,driver,小黄,19,上海,120
7,28,driver,小黄,25,深圳,123456
8,18,演员,小黄,19,上海,120
9,18,演员,小黄,25,深圳,123456


In [113]:
df1

,age,job,name
a,18,student,小明
b,28,driver,小黄
c,30,seller,小绿
d,40,boss,小白


In [118]:
df6 = DataFrame({
    "姓名":["小明","小黄","小绿","小白","小明","小绿","小明","小黄"],
    "家乡":["北京","上海","杭州","新加坡","日本","韩国","美国","深圳"],
    "电话":[110,120,119,999,10086,10010,4004000,"小明"]
},index=list("abcdefgh"))
df6

,姓名,家乡,电话
a,小明,北京,110
b,小黄,上海,120
c,小绿,杭州,119
d,小白,新加坡,999
e,小明,日本,10086
f,小绿,韩国,10010
g,小明,美国,4004000
h,小黄,深圳,小明


In [119]:
df1.merge(df6,left_on="name",right_on="姓名")
# 没有公共的属性时，可以采用left_on和right_on去指定左侧和右侧出那个属性来合并
# 由于左右不是公共属性，不能并在一起

,age,job,name,姓名,家乡,电话
0,18,student,小明,小明,北京,110
1,18,student,小明,小明,日本,10086
2,18,student,小明,小明,美国,4004000
3,28,driver,小黄,小黄,上海,120
4,28,driver,小黄,小黄,深圳,小明
5,30,seller,小绿,小绿,杭州,119
6,30,seller,小绿,小绿,韩国,10010
7,40,boss,小白,小白,新加坡,999


In [121]:
df1.merge(df6,left_on="name",right_on="家乡")

,age,job,name,姓名,家乡,电话


============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

推广：left_index和right_index

In [135]:
df1.merge(df6,left_index=True,right_index=True,how="outer")

,age,job,name,姓名,家乡,电话
a,18.0,student,小明,小明,北京,110
b,28.0,driver,小黄,小黄,上海,120
c,30.0,seller,小绿,小绿,杭州,119
d,40.0,boss,小白,小白,新加坡,999
e,NaN,NaN,NaN,小明,日本,10086
f,NaN,NaN,NaN,小绿,韩国,10010
g,NaN,NaN,NaN,小明,美国,4004000
h,NaN,NaN,NaN,小黄,深圳,小明


In [139]:
df1.merge(df6,left_index=True,right_on="姓名",how="inner")

,age,job,name,姓名,家乡,电话


### 5) 内合并与外合并

In [122]:
df1

,age,job,name
a,18,student,小明
b,28,driver,小黄
c,30,seller,小绿
d,40,boss,小白


In [124]:
df7 = DataFrame({
    "name":["小明","小黄","小绿","小白","Jack","安倍","Obama","李嘉诚"],
    "home":["北京","上海","杭州","新加坡","日本","韩国","美国","深圳"],
    "phone":[110,120,119,999,10086,10010,4004000,"小明"]
},index=list("abcdefgh"))
df7

,home,name,phone
a,北京,小明,110
b,上海,小黄,120
c,杭州,小绿,119
d,新加坡,小白,999
e,日本,Jack,10086
f,韩国,安倍,10010
g,美国,Obama,4004000
h,深圳,李嘉诚,小明


In [125]:
df1.merge(df7)

,age,job,name,home,phone
0,18,student,小明,北京,110
1,28,driver,小黄,上海,120
2,30,seller,小绿,杭州,119
3,40,boss,小白,新加坡,999


- 内合并：只保留两者都有的key（默认模式）

In [126]:
df1.merge(df7,how="inner")
# how参数代表的是合并的方式默认是"inner"代表内合并
# 两个表里面公共部分保留，不匹配的部分删除

,age,job,name,home,phone
0,18,student,小明,北京,110
1,28,driver,小黄,上海,120
2,30,seller,小绿,杭州,119
3,40,boss,小白,新加坡,999


- 外合并：所有的值全部并在一起，不匹配的地方补NaN

In [127]:
df1.merge(df7,how="outer")

,age,job,name,home,phone
0,18.0,student,小明,北京,110
1,28.0,driver,小黄,上海,120
2,30.0,seller,小绿,杭州,119
3,40.0,boss,小白,新加坡,999
4,NaN,NaN,Jack,日本,10086
5,NaN,NaN,安倍,韩国,10010
6,NaN,NaN,Obama,美国,4004000
7,NaN,NaN,李嘉诚,深圳,小明


- 左合并、右合并：how='left'，how='right'，

In [129]:
df1.merge(df7,how="right")
# left左合并，以左侧表的值为基准，右有左侧没有的地方删除，左侧有右侧没有的地方补Nan
# right右合并，以右侧为基准

,age,job,name,home,phone
0,18.0,student,小明,北京,110
1,28.0,driver,小黄,上海,120
2,30.0,seller,小绿,杭州,119
3,40.0,boss,小白,新加坡,999
4,NaN,NaN,Jack,日本,10086
5,NaN,NaN,安倍,韩国,10010
6,NaN,NaN,Obama,美国,4004000
7,NaN,NaN,李嘉诚,深圳,小明


============================================

练习17：

1. 如果只有张三赵小六语数英三个科目的成绩，如何合并？

3. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

## 作业
## 3. 案例分析：美国各州人口数据分析

要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息

In [140]:
# 导入数据
abb = pd.read_csv("./usapop/state-abbrevs.csv")
areas = pd.read_csv("./usapop/state-areas.csv")
pop = pd.read_csv("./usapop/state-population.csv")

In [141]:
abb

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [142]:
areas

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [143]:
pop

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
5,AL,total,2011,4801627.0
6,AL,total,2009,4757938.0
7,AL,under18,2009,1134192.0
8,AL,under18,2013,1111481.0
9,AL,total,2013,4833722.0


###### 对数据进行整合与清洗

1、合并pop和abb两张表

In [179]:
pop2 = abb.merge(pop,left_on="abbreviation",right_on="state/region",how="outer")

In [180]:
pop2.head(5) # head是输出前n行，默认是5

,state,abbreviation,state/region,ages,year,population
0,Alabama,AL,AL,under18,2012,1117489.0
1,Alabama,AL,AL,total,2012,4817528.0
2,Alabama,AL,AL,under18,2010,1130966.0
3,Alabama,AL,AL,total,2010,4785570.0
4,Alabama,AL,AL,under18,2011,1125763.0


In [181]:
pop2

,state,abbreviation,state/region,ages,year,population
0,Alabama,AL,AL,under18,2012,1117489.0
1,Alabama,AL,AL,total,2012,4817528.0
2,Alabama,AL,AL,under18,2010,1130966.0
3,Alabama,AL,AL,total,2010,4785570.0
4,Alabama,AL,AL,under18,2011,1125763.0
5,Alabama,AL,AL,total,2011,4801627.0
6,Alabama,AL,AL,total,2009,4757938.0
7,Alabama,AL,AL,under18,2009,1134192.0
8,Alabama,AL,AL,under18,2013,1111481.0
9,Alabama,AL,AL,total,2013,4833722.0


In [182]:
# 删除掉一列简称
pop2.drop("abbreviation",axis=1,inplace=True)
# inplace参数表示是否在原表上修改，True在原表上修改
# False默认，不在原表上修改，生成一个新表返回出来

查看数据的缺失情况

In [183]:
pop2.isnull().any()

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

通过上面的检测，发现state属性和population属性有数据缺失

接下来我们具体来查看缺失情况

In [184]:
# state的缺失
cond_state = pop2["state"].isnull()
pop2[cond_state]["state/region"].unique() # 输出缺失的那些周的行

array(['PR', 'USA'], dtype=object)

我们发现PR和USA这两个简称对应的state没有

通过分析发现PR代表Puerto Rico，USA代表United States

In [185]:
# 找到简称为PR的那些行
cond_pr = pop2["state/region"] == "PR"
pop2[cond_pr] 

,state,state/region,ages,year,population
2448,NaN,PR,under18,1990,NaN
2449,NaN,PR,total,1990,NaN
2450,NaN,PR,total,1991,NaN
2451,NaN,PR,under18,1991,NaN
2452,NaN,PR,total,1993,NaN
2453,NaN,PR,under18,1993,NaN
2454,NaN,PR,under18,1992,NaN
2455,NaN,PR,total,1992,NaN
2456,NaN,PR,under18,1994,NaN
2457,NaN,PR,total,1994,NaN


In [188]:
pop2["state"][cond_pr] = "Puerto Rico" 

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [189]:
pop2[cond_pr]

,state,state/region,ages,year,population
2448,Puerto Rico,PR,under18,1990,NaN
2449,Puerto Rico,PR,total,1990,NaN
2450,Puerto Rico,PR,total,1991,NaN
2451,Puerto Rico,PR,under18,1991,NaN
2452,Puerto Rico,PR,total,1993,NaN
2453,Puerto Rico,PR,under18,1993,NaN
2454,Puerto Rico,PR,under18,1992,NaN
2455,Puerto Rico,PR,total,1992,NaN
2456,Puerto Rico,PR,under18,1994,NaN
2457,Puerto Rico,PR,total,1994,NaN


In [191]:
# 填补USA
pop2["state"][pop2["state/region"] == "USA"] = "United States"

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [193]:
pop2.isnull().any()

state           False
state/region    False
ages            False
year            False
population       True
dtype: bool

处理人口缺失问题

In [196]:
cond_pop = pop2["population"].isnull()

In [197]:
pop2[cond_pop]

,state,state/region,ages,year,population
2448,Puerto Rico,PR,under18,1990,NaN
2449,Puerto Rico,PR,total,1990,NaN
2450,Puerto Rico,PR,total,1991,NaN
2451,Puerto Rico,PR,under18,1991,NaN
2452,Puerto Rico,PR,total,1993,NaN
2453,Puerto Rico,PR,under18,1993,NaN
2454,Puerto Rico,PR,under18,1992,NaN
2455,Puerto Rico,PR,total,1992,NaN
2456,Puerto Rico,PR,under18,1994,NaN
2457,Puerto Rico,PR,total,1994,NaN


In [199]:
pop2.dropna(inplace=True)

In [201]:
pop2.isnull().any()

state           False
state/region    False
ages            False
year            False
population      False
dtype: bool

合并面积areas表

In [206]:
pop3 = pop2.merge(areas,how="outer")
pop3

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0
5,Alabama,AL,total,2011,4801627.0,52423.0
6,Alabama,AL,total,2009,4757938.0,52423.0
7,Alabama,AL,under18,2009,1134192.0,52423.0
8,Alabama,AL,under18,2013,1111481.0,52423.0
9,Alabama,AL,total,2013,4833722.0,52423.0


USA的面积缺失，通过分析可以加出来

In [205]:
usa_area = areas["area (sq. mi)"].sum()
usa_area

3790399

In [209]:
pop3["area (sq. mi)"][pop3["state/region"] == "USA"] = usa_area

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [211]:
pop3.isnull().any()

state            False
state/region     False
ages             False
year             False
population       False
area (sq. mi)    False
dtype: bool

经过以上的处理，数据已经整合并且清洗完毕

接下来对pop3中的数据进行分析与挖掘

1、查询2012年的每个州的全民人口数据

In [212]:
pop3.head()

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0


In [214]:
pop_2012 = pop3.query("ages=='total' & year==2012")
pop_2012

,state,state/region,ages,year,population,area (sq. mi)
1,Alabama,AL,total,2012,4817528.0,52423.0
95,Alaska,AK,total,2012,730307.0,656425.0
97,Arizona,AZ,total,2012,6551149.0,114006.0
191,Arkansas,AR,total,2012,2949828.0,53182.0
193,California,CA,total,2012,37999878.0,163707.0
287,Colorado,CO,total,2012,5189458.0,104100.0
289,Connecticut,CT,total,2012,3591765.0,5544.0
383,Delaware,DE,total,2012,917053.0,1954.0
385,District of Columbia,DC,total,2012,633427.0,68.0
479,Florida,FL,total,2012,19320749.0,65758.0


2、对查询下结果处理，用state作为新的索引

In [216]:
pop_2012.set_index("state",inplace=True)

In [217]:
pop_2012

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2012,4817528.0,52423.0
Alaska,AK,total,2012,730307.0,656425.0
Arizona,AZ,total,2012,6551149.0,114006.0
Arkansas,AR,total,2012,2949828.0,53182.0
California,CA,total,2012,37999878.0,163707.0
Colorado,CO,total,2012,5189458.0,104100.0
Connecticut,CT,total,2012,3591765.0,5544.0
Delaware,DE,total,2012,917053.0,1954.0
District of Columbia,DC,total,2012,633427.0,68.0


In [219]:
pop_2012["state"] # state现在是索引，而不是属性

KeyError: 'state'

3、计算各州的2012年人口密度

In [223]:
des = pop_2012["population"] / pop_2012["area (sq. mi)"]
des

state
Alabama                   91.897221
Alaska                     1.112552
Arizona                   57.463195
Arkansas                  55.466662
California               232.121278
Colorado                  49.850701
Connecticut              647.865260
Delaware                 469.320880
District of Columbia    9315.102941
Florida                  293.815946
Georgia                  166.814926
Hawaii                   127.157885
Idaho                     19.091942
Illinois                 222.179495
Indiana                  179.510763
Iowa                      54.642103
Kansas                    35.067184
Kentucky                 108.379649
Louisiana                 88.770596
Maine                     37.542063
Montana                    6.837955
Nebraska                  23.983945
Nevada                    24.911176
New Hampshire            141.334296
New Jersey              1016.710502
New Mexico                17.135361
New York                 359.359798
North Carolina        

In [228]:
des_df = DataFrame(des,columns=["densey (人/sq.mi)"])
des_df

,densey (人/sq.mi)
state,
Alabama,91.897221
Alaska,1.112552
Arizona,57.463195
Arkansas,55.466662
California,232.121278
Colorado,49.850701
Connecticut,647.865260
Delaware,469.320880
District of Columbia,9315.102941


In [232]:
# 把人口密度拼接到pop_2012上面
pop_2012_dens = pd.merge(pop_2012,des_df,left_index=True,right_index=True)

In [233]:
pop_2012_dens.head()

,state/region,ages,year,population,area (sq. mi),densey (人/sq.mi)
state,,,,,,
Alabama,AL,total,2012,4817528.0,52423.0,91.897221
Alaska,AK,total,2012,730307.0,656425.0,1.112552
Arizona,AZ,total,2012,6551149.0,114006.0,57.463195
Arkansas,AR,total,2012,2949828.0,53182.0,55.466662
California,CA,total,2012,37999878.0,163707.0,232.121278


4、排序、并且求密度最高的五个州和密度最低的五个州

In [235]:
pop_2012_dens.sort_values("densey (人/sq.mi)",inplace=True)

In [236]:
pop_2012_dens

,state/region,ages,year,population,area (sq. mi),densey (人/sq.mi)
state,,,,,,
Alaska,AK,total,2012,730307.0,656425.0,1.112552
Wyoming,WY,total,2012,576626.0,97818.0,5.894886
Montana,MT,total,2012,1005494.0,147046.0,6.837955
North Dakota,ND,total,2012,701345.0,70704.0,9.919453
South Dakota,SD,total,2012,834047.0,77121.0,10.814785
New Mexico,NM,total,2012,2083540.0,121593.0,17.135361
Idaho,ID,total,2012,1595590.0,83574.0,19.091942
Nebraska,NE,total,2012,1855350.0,77358.0,23.983945
Nevada,NV,total,2012,2754354.0,110567.0,24.911176


In [237]:
pop_2012_dens.head(5)

,state/region,ages,year,population,area (sq. mi),densey (人/sq.mi)
state,,,,,,
Alaska,AK,total,2012,730307.0,656425.0,1.112552
Wyoming,WY,total,2012,576626.0,97818.0,5.894886
Montana,MT,total,2012,1005494.0,147046.0,6.837955
North Dakota,ND,total,2012,701345.0,70704.0,9.919453
South Dakota,SD,total,2012,834047.0,77121.0,10.814785


In [239]:
pop_2012_dens.tail().loc[::-1]

,state/region,ages,year,population,area (sq. mi),densey (人/sq.mi)
state,,,,,,
District of Columbia,DC,total,2012,633427.0,68.0,9315.102941
Puerto Rico,PR,total,2012,3651545.0,3515.0,1038.846373
New Jersey,NJ,total,2012,8867749.0,8722.0,1016.710502
Rhode Island,RI,total,2012,1050304.0,1545.0,679.808414
Connecticut,CT,total,2012,3591765.0,5544.0,647.865260


## 回顾：Series/DataFrame运算与ndarray运算的区别

- Series与DataFrame没有广播，如果对应index没有值，则记为NaN；或者使用add的fill_value来补缺失值
- ndarray有广播，通过重复已有值来计算